# WorkFlow

1.Import the Important libraries<br>
2.Load the dataset (convert into tensors) -> (convert to batches)<br>
3.Create the Logistic Regression Model<br>
4.Pick a loss function and optimizer<br>
5.Train the model in loop<br>
6.Evaluate the model<br>
7.Improve the model<br>
8.save the model<br>

### 1.Importing important Libraries

In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
import matplotlib.pyplot as plt

### 2.Load the Data(convert into Tensors)

In [ ]:
train_dataset=datasets.FashionMNIST(root='datasets',train=True,transform=transforms.ToTensor(),download=True)
test_dataset=datasets.FashionMNIST(root='datasets',train=False,transform=transforms.ToTensor())

In [ ]:
train_dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: datasets
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
test_dataset

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: datasets
    Split: Test
    StandardTransform
Transform: ToTensor()

### 2.2 Converting dataset into batch

In [ ]:
batch_size=64

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

### 3.Build Logistic Regression Model

In [ ]:
class LogisticRegression(nn.Module):
  def __init__(self, in_dim, classes) -> None:
    super().__init__()
    self.linear = nn.Linear(in_dim,classes)

  def forward(self,X):
    out = self.linear(X)
    return out

In [ ]:
model = LogisticRegression(28*28,10)

In [ ]:
model.state_dict()

OrderedDict([('linear.weight',
              tensor([[ 2.8608e-02,  1.2115e-02,  1.3590e-02,  ...,  3.8286e-03,
                        6.2245e-03, -2.4749e-02],
                      [-1.5689e-02, -2.2931e-03,  2.4451e-02,  ..., -3.5455e-02,
                        2.9486e-02,  1.1207e-02],
                      [-3.4834e-02, -2.2571e-02, -5.1539e-03,  ...,  1.9972e-02,
                        2.8091e-03, -3.1551e-02],
                      ...,
                      [ 3.0142e-02,  3.4225e-02,  4.7370e-04,  ...,  2.0819e-02,
                       -3.2220e-02,  1.3310e-02],
                      [ 7.5055e-05,  2.7403e-02, -2.3926e-02,  ...,  2.7981e-02,
                       -1.7780e-02, -5.4094e-03],
                      [-1.8836e-02, -1.9098e-02,  2.3573e-02,  ..., -1.5692e-02,
                       -7.8596e-03,  1.7541e-02]])),
             ('linear.bias',
              tensor([-0.0292,  0.0319, -0.0057,  0.0133,  0.0130,  0.0050,  0.0017, -0.0031,
                       0.0275,

In [ ]:
device = 'cude' if torch.cuda.is_available() else 'cpu'

In [ ]:
device

'cpu'

In [ ]:
model = model.to(device)

### 4.Pick a loss function and optimizer

In [ ]:
learning_rate = 1e-3

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

### 5.Building a Training Loop

In [ ]:
total_epochs = 10

In [ ]:
for epoch in range(total_epochs):
  print("----" * 15)
  print(f"epochs : {epoch + 1}")
  since = time.time()
  running_loss=0.0
  running_accuracy=0.0
  model.train()
  for i , data in enumerate(train_loader,1):
    img,label=data
    img = img.view(img.size(0),-1)
    img = img.to(device)
    label = label.to(device)

    out=model(img)
    loss=criterion(out,label)
    running_loss += loss.item()
    _,pred = torch.max(out,1)
    running_accuracy += (label == pred).float().mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i % 300 == 0: #for printing losses ,print out what is happing every 300 indexing
        print(f'[{epoch+1}/{total_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_accuracy/i:.6f}')
  print(f'Finish {epoch+1} epoch, Loss: {running_loss/i:.6f}, Acc: {running_accuracy/i:.6f}')

------------------------------------------------------------
epochs : 1
[1/10] Loss: 0.996800, Acc: 0.693750
[1/10] Loss: 0.982216, Acc: 0.697604
[1/10] Loss: 0.968849, Acc: 0.700781
Finish 1 epoch, Loss: 0.966921, Acc: 0.701276
------------------------------------------------------------
epochs : 2
[2/10] Loss: 0.904958, Acc: 0.716302
[2/10] Loss: 0.902944, Acc: 0.716458
[2/10] Loss: 0.895753, Acc: 0.718542
Finish 2 epoch, Loss: 0.895453, Acc: 0.718750
------------------------------------------------------------
epochs : 3
[3/10] Loss: 0.859657, Acc: 0.731875
[3/10] Loss: 0.852594, Acc: 0.733359
[3/10] Loss: 0.847344, Acc: 0.734913
Finish 3 epoch, Loss: 0.846488, Acc: 0.734758
------------------------------------------------------------
epochs : 4
[4/10] Loss: 0.823873, Acc: 0.740885
[4/10] Loss: 0.814806, Acc: 0.742734
[4/10] Loss: 0.810120, Acc: 0.745434
Finish 4 epoch, Loss: 0.810085, Acc: 0.745253
------------------------------------------------------------
epochs : 5
[5/10] Loss:

### 6.Model Evaluation

In [ ]:
model.eval()
eval_loss = 0.0
eval_accuracy = 0.0
for data in test_loader:
  img,label = data
  img = img.view(img.size(0),-1)
  img = img.to(device)
  label = label.to(device)
  with torch.no_grad():
    out = model(img)
    loss = criterion(out,label)
  eval_loss += loss
  _,predict = torch.max(out,1)
  eval_accuracy += (predict == label).float().mean()
print(f'Test Loss: {eval_loss/len(test_loader):.6f}, Acc: {eval_accuracy/len(test_loader):.6f}')
print(f'Time:{(time.time()-since):.1f} s')

Test Loss: 0.704714, Acc: 0.770701
Time:423.0 s


In [ ]:
model.state_dict()

OrderedDict([('linear.weight',
              tensor([[ 2.8607e-02,  1.2151e-02,  1.3816e-02,  ..., -1.3907e-03,
                        4.5750e-03, -2.4840e-02],
                      [-1.5691e-02, -2.2952e-03,  2.4424e-02,  ..., -3.6627e-02,
                        2.9135e-02,  1.1162e-02],
                      [-3.4838e-02, -2.2572e-02, -5.1651e-03,  ...,  2.7631e-02,
                        3.8289e-03, -3.1398e-02],
                      ...,
                      [ 3.0142e-02,  3.4222e-02,  4.5728e-04,  ...,  1.8833e-02,
                       -3.2870e-02,  1.3249e-02],
                      [ 7.4085e-05,  2.7393e-02, -2.4036e-02,  ...,  2.4268e-02,
                       -1.8722e-02, -5.4740e-03],
                      [-1.8837e-02, -1.9101e-02,  2.3550e-02,  ..., -1.5294e-02,
                       -7.2748e-03,  1.7614e-02]])),
             ('linear.bias',
              tensor([-0.0251,  0.0346, -0.0513,  0.0041, -0.1311,  0.3642,  0.0335,  0.0125,
                      -0.0652,

In [ ]:
torch.save(model.state_dict(),"./mymodel.pth")